In [1]:
import netCDF4
import os
import xarray as xr

In [3]:
# load the NetCDF file (NetCDF4 library needed)
path_dataset = '../data/cmems_20200916.nc'
path_dataset = '../data/gfs_20200916.nc'
assert os.path.isfile(path_dataset)
cmems = xr.open_dataset(path_dataset)

OSError: [Errno -51] NetCDF: Unknown file format: b'/DATA1/SHARED/ocean-route-optimization/data/gfs_20200916.nc'

In [ ]:
# index the variable u, time 2020-09-16 and lon-lat (-10, 10) x (-20, 20)
u = cmems['u'].sel(time='2020-09-16', lon=slice(-10, 10), lat=slice(-20, 20))